In [27]:
import os, re
import datetime
import dateutil

import pandas as pd
import plotly.express as px

## Generate `generated_schedule.qmd`

In [28]:
safe_mkdir = lambda x: None if os.path.exists(x) else os.mkdir(x) 

# setup output structure
safe_mkdir('./generated')
safe_mkdir('./generated/talks')

In [29]:
# make a nice looking schedule from the massaged data
schedule = pd.read_excel('./data/003_MGM2024_schedule.xlsx')
schedule

,Day,Session,Chair,Time,Name,Presenter,Link,Location
0,"Wednesday, February 28, 2024",NaN,NaN,8:00 AM – 6:00 PM,OPTIONAL PRE-CONFERENCE WORKSHOPS,NaN,NaN,NaN
1,"Wednesday, February 28, 2024",NaN,NaN,8:00 AM - 12:00 PM,Maize Crop Germplasm Committee,NaN,NaN,Room 302C
2,"Wednesday, February 28, 2024",NaN,NaN,12:00 PM – 4:00 PM,Corn Breeding Research,NaN,NaN,Room 302A/B
3,"Thursday, February 29, 2024",NaN,NaN,9:00 AM – 6:00 PM,OPTIONAL PRE-CONFERENCE WORKSHOPS,NaN,NaN,NaN
4,"Thursday, February 29, 2024",NaN,NaN,8:00 AM – 4:00 PM,Corn Breeding Research,NaN,NaN,Rooms 302A/B
...,...,...,...,...,...,...,...,...
91,"Sunday, March 3, 2024",SESSION 11 – METABOLIC REGULATION,Chair: Sherry Flint-Garcia,10:20:00,Upper level and cross hierarchical regulation ...,"Ankita Abnave, University of Toledo",[T28],NaN
92,"Sunday, March 3, 2024",SESSION 11 – METABOLIC REGULATION,Chair: Sherry Flint-Garcia,10:40:00,Elucidating the metabolomic and single-cell tr...,"Manwinder Singh Brar, Clemson University",[T29],NaN
93,"Sunday, March 3, 2024",SESSION 11 – METABOLIC REGULATION,Chair: Sherry Flint-Garcia,11:00:00,Potent pollen gene regulation by DNA glycosyla...,"Jonathan Gent, University of Georgia",[T30],NaN
94,"Sunday, March 3, 2024",NaN,NaN,11:20:00,CLOSING REMARKS,Rubén Rellán Álvarez and Sherry Flint-Garcia,NaN,NaN


In [30]:
schedule['StartTime'] = None
schedule['EndTime']   = None

In [31]:
for i in range(len(schedule)):
    day = schedule.loc[i, 'Day']
    x = schedule.loc[i, 'Time']
    if type(x) == str:
        if ('–' in x) or ('-' in x):
            x = x.split('–') if '–' in x else x.split('-')
            x = [e.strip() for e in x]
            schedule.loc[i, 'StartTime'] = dateutil.parser.parse(day+' '+x[0])
            schedule.loc[i, 'EndTime']   = dateutil.parser.parse(day+' '+x[0])
        else:
            schedule.loc[i, 'StartTime'] = dateutil.parser.parse(day+' '+x.strip())
    elif type(x) == datetime.time:
        schedule.loc[i, 'StartTime'] = datetime.datetime.combine(dateutil.parser.parse(day), x)
    else:
        print(x)
    

In [32]:
## Convert Time -> StartTime	EndTime

add_end_idx = schedule.loc[(schedule.EndTime.isna()), ].index

for i in add_end_idx:
    if i == schedule.index.max():
        # last entry
        schedule.loc[i, 'EndTime'] = schedule.loc[i, 'StartTime']
    else:
        # not last entry
        if schedule.loc[i, 'Day'] == schedule.loc[i+1, 'Day']:
            schedule.loc[i, 'EndTime'] = schedule.loc[i+1, 'StartTime']
        else: 
            # end of day
            schedule.loc[i, 'EndTime'] = schedule.loc[i, 'StartTime']

schedule = schedule.drop(columns='Time')

In [33]:
## fix chair info
schedule['Chair'] = [e.replace('Chair:', '').strip() if type(e) == str else e for e in schedule.Chair.tolist()]

In [34]:
# # option 1 formatted text
# out = []
# current = {e:'' for e in ['Day', 'Session', 'Chair']}

# # for i in range(20):
# for i in schedule.index:
#     d = schedule.loc[i, ].to_dict()

#     if not current['Day'] == d['Day']:
#         if out != []:
#             out.append('\n\n')
#         out.append(f"## {d['Day']}")
#         current['Day'] = d['Day']

#     if not current['Session'] == d['Session']:
#         if not pd.isna(d['Session']):
#             out.append(f"***{d['Session']}***")
#             out.append(f"**{d['Chair']}**")
#         current['Session'] = d['Session']
#         current['Chair'] = d['Chair']

#     str_time = f"{ d['StartTime'].strftime('%H:%M') } - { d['EndTime'].strftime('%H:%M') }"
    
#     x_or_blank = lambda x : x if not pd.isna(x) else ''
    
#     str_name = x_or_blank(d['Name'])
#     str_pres = x_or_blank(d['Presenter'])
#     str_link = x_or_blank(d['Link'])

#     str_loc = x_or_blank(d['Location'])

#     out.append(f"{str_time} {str_name}"+"\n"+f"{str_pres} {str_link} {str_loc} ")

# # print('\n'.join(out))

In [35]:
# option 2 markdown tables
out = []
current = {e:'' for e in ['Day', 'Session', 'Chair']}

table_header = """
|  |        |   |  
|:-|--------|--:|"""
# for i in range(20):
for i in schedule.index:
    d = schedule.loc[i, ].to_dict()

    if not current['Day'] == d['Day']:
        if out != []:
            out.append('\n\n')
        out.append(f"## {d['Day']}")
        current['Day'] = d['Day']

    if not ((current['Session'] == d['Session']) or
            (pd.isna(current['Session']) == pd.isna(d['Session']))):
        if not pd.isna(d['Session']):
            out.append('\n')
            out.append(f"### {d['Session']}")
            out.append(f"\n**{d['Chair']}**")
        current['Session'] = d['Session']
        current['Chair'] = d['Chair']

    

    str_time = f"{ d['StartTime'].strftime('%H:%M') } - { d['EndTime'].strftime('%H:%M') }"
    
    x_or_blank = lambda x : x if not pd.isna(x) else ''
    
    str_name = x_or_blank(d['Name'])
    str_pres = x_or_blank(d['Presenter'])
    str_link = x_or_blank(d['Link'])

    # convert to link 
    str_link = str_link if str_link == '' else str_link
    if str_link != '':
        if re.match('\[.+\]', str_link):
            str_link = f"{str_link}(/generated/talks/{str_link.replace('[', '').replace(']', '')}/index.qmd)"

    str_loc = x_or_blank(d['Location'])

    if out[-1][0] != '|':
        out.append(table_header)
    out.append(f"| {str_time} | {str_name} | {str_pres} {str_link} {str_loc} |")

out = '\n'.join(out)

In [36]:
qmd_content = """---
title: "Schedule of Events"
---

::: {.callout-note appearance="simple" icon="false"}
## Note:

Talks will be held in the Ballroom A Posters will be displayed in the Ballroom B&C
:::

"""+out

with open('./generated_schedule.qmd', 'w') as f:
    f.writelines(qmd_content)

In [37]:
schedule.to_csv('./generated/schedule.csv', index=None)

## Set up Structure

In [38]:
safe_mkdir = lambda x: None if os.path.exists(x) else os.mkdir(x) 

# setup output structure
safe_mkdir('./generated')
safe_mkdir('./generated/talks')

qmd_content = """---
title: "Presentation Abstracts"
listing:
  contents:
    - "./"
  sort: "date"
  type: grid
  categories: true
  sort-ui: false
  filter-ui: false
page-layout: full
title-block-banner: true
editor: source
---
"""

with open('./generated/talks/index.qmd', 'w') as f:
    f.writelines(qmd_content)

In [39]:
data = {e:pd.read_excel(
    './data/MM2024-data.xlsx',
    sheet_name=e) for  e in ['Posters', 'Talks', 'Author Index', 'Attendee List']}

### Write out Talks

In [40]:
Talks = data['Talks']

out = []

callout = '::: {.callout-note collapse="true"}'
callout_dots = ':::'

for i in Talks.index:
    d = Talks.loc[i, ].to_dict()

    out.append( f"""
## {d['TALK']}: {d['TITLE']}
{d['CATEGORY']}
[{d['PRESENTER_FIRST_NAME']} {d['PRESENTER_LAST_NAME']}](mailto:{d['PRESENTER_EMAIL']})

{callout}
## Author List
{d['AUTHOR_LIST']}

{d['AFFILIATION_LIST']}
{callout_dots}

{callout}
## Abstract

{d['ABSTRACT']}
{callout_dots}
"""
    )


In [41]:
qmd_content = """---
title: "Presentations"
---"""+''.join(out)

with open('./generated_talks.qmd', 'w') as f:
    f.writelines(qmd_content)

## Repeat with posters

In [42]:
Posters = data['Posters']

out = []

callout = '::: {.callout-note collapse="true"}'
callout_dots = ':::'

for i in Posters.index:
    d = Posters.loc[i, ].to_dict()

    out.append( f"""
## {d['POSTER']}: {d['TITLE']}
{d['CATEGORY']}
[{d['PRESENTER_FIRST_NAME']} {d['PRESENTER_LAST_NAME']}](mailto:{d['PRESENTER_EMAIL']}) ({d['PRESENTER_ROLE']})

{callout}
## Author List
{d['AUTHOR_LIST']}

{d['AFFILIATION_LIST']}
{callout_dots}

{callout}
## Abstract

{d['ABSTRACT']}
{callout_dots}
"""
    )


In [43]:
qmd_content = """---
title: "Posters"
---"""+''.join(out)

with open('./generated_poster_list.qmd', 'w') as f:
    f.writelines(qmd_content)

## Write out presentation/poster pages

In [44]:
# setup output structure
safe_mkdir('./generated')
safe_mkdir('./generated/posters')
safe_mkdir('./generated/talks')

# write index qmd pages
qmd_content = """---
title: "Posters"
listing:
  contents:
    - "./"
  type: grid
  categories: true
  sort-ui: false
  filter-ui: false
page-layout: full
title-block-banner: true
editor: source
---
"""

with open('./generated/posters/index.qmd', 'w') as f:
    f.writelines(qmd_content)

qmd_content = """---
title: "Presentation Abstracts"
listing:
  contents:
    - "./"
  sort: "date"
  type: grid
  categories: true
  sort-ui: false
  filter-ui: false
page-layout: full
title-block-banner: true
editor: source
---
"""

with open('./generated/talks/index.qmd', 'w') as f:
    f.writelines(qmd_content)

In [45]:
category_2_image = {
    'Computational and Large-Scale Biology': 'https://upload.wikimedia.org/wikipedia/commons/e/e5/ENIAC-changing_a_tube.jpg',
    'Cell and Developmental Biology': 'https://upload.wikimedia.org/wikipedia/commons/c/c1/Protective_Coleoptile_Tip_in_Zea_Mays_Embryo_%2847691344541%29.jpg',
    'Biochemical and Molecular Genetics': 'https://upload.wikimedia.org/wikipedia/commons/0/05/DNA_base-pair_diagram.jpg',
    'Transposons & Epigenetics': 'https://upload.wikimedia.org/wikipedia/commons/b/b3/Genetic_corn.jpg',
    'Quantitative Genetics & Breeding': 'https://upload.wikimedia.org/wikipedia/commons/5/5e/Aztec_Diet_Basket_of_Corn_%289755332575%29.jpg',
    'Education & Outreach': 'https://upload.wikimedia.org/wikipedia/commons/6/69/Leiden_University_Library_-_Seikei_Zusetsu_vol._19%2C_page_019_-_%E7%8E%89%E8%9C%80%E9%BB%8D_-_Zea_mays_L.%2C_1804.jpg',
    'Cytogenetics': 'https://upload.wikimedia.org/wikipedia/commons/a/ab/Angiosperm_Morphology_The_Closed_and_Collateral_Vascular_Bundle_in_Zea_%2837215761790%29.jpg',
    'Evolution and Population Genetics': 'https://upload.wikimedia.org/wikipedia/commons/9/9a/CollapsedtreeLabels-simplified-fi.svg'
}

In [46]:
# Write poster presentation info ----

for i in data['Posters'].index:
    d = data['Posters'].loc[i, ].to_dict()

    # if we have a multi-category case":
    # import re
    # categories = re.findall(r'>.+<', d['CATEGORY'])
    # categories = [e.replace('>', '').replace('<', '').strip() for e in categories]
    # categories

    qmd_header = f"""---
title: "{d['TITLE']}"
subtitle: "{d['POSTER']}"
author: "{d['PRESENTER_FIRST_NAME']+' '+d['PRESENTER_LAST_NAME']}"
date: ""
image: "{category_2_image[d['CATEGORY'].split('>')[1].split('<')[0].strip()]}"
categories: 
- {d['CATEGORY'].split('>')[1].split('<')[0].strip()}
- {'Even Numbered' if (int(d['POSTER'].replace('P', '')) % 2 == 0) else 'Odd Numbered'}
freeze: true
editor: source
markdown: 
    wrap: 72
---"""

    qmd_content = '\n\n'.join([
        qmd_header,
        d['ABSTRACT'],
        d['AUTHOR_LIST'],
        d['AFFILIATION_LIST']])

    save_to = f"./generated/posters/{d['POSTER']}/index.qmd"

    # make output dir if missing
    safe_mkdir('/'.join(save_to.split('/')[0:-1]))

    with open(save_to, 'w') as f:
        f.writelines(qmd_content)

In [47]:
# Write talk presentation info ----

for i in data['Talks'].index:
    d = data['Talks'].loc[i, ].to_dict()

    qmd_header = f"""---
title: "{d['TITLE']}"
subtitle: "{d['TALK']}"
author: "{d['PRESENTER_FIRST_NAME']+' '+d['PRESENTER_LAST_NAME']}"
date: ""
image: "{category_2_image[d['CATEGORY'].split('>')[1].split('<')[0].strip()]}"
categories: 
- {d['CATEGORY'].split('>')[1].split('<')[0].strip()}
freeze: true
editor: source
markdown: 
    wrap: 72
---"""

    qmd_content = '\n\n'.join([
        qmd_header,
        d['ABSTRACT'],
        d['AUTHOR_LIST'],
        d['AFFILIATION_LIST']])

    save_to = f"./generated/talks/{d['TALK']}/index.qmd"

    # make output dir if missing
    safe_mkdir('/'.join(save_to.split('/')[0:-1]))

    with open(save_to, 'w') as f:
        f.writelines(qmd_content)

## Write out presentation/poster pages

In [48]:
# category_2_image = {
#     'Computational and Large-Scale Biology': 'https://upload.wikimedia.org/wikipedia/commons/e/e5/ENIAC-changing_a_tube.jpg',
#     'Cell and Developmental Biology': 'https://upload.wikimedia.org/wikipedia/commons/c/c1/Protective_Coleoptile_Tip_in_Zea_Mays_Embryo_%2847691344541%29.jpg',
#     'Biochemical and Molecular Genetics': 'https://upload.wikimedia.org/wikipedia/commons/0/05/DNA_base-pair_diagram.jpg',
#     'Transposons & Epigenetics': 'https://upload.wikimedia.org/wikipedia/commons/b/b3/Genetic_corn.jpg',
#     'Quantitative Genetics & Breeding': 'https://upload.wikimedia.org/wikipedia/commons/5/5e/Aztec_Diet_Basket_of_Corn_%289755332575%29.jpg',
#     'Education & Outreach': 'https://upload.wikimedia.org/wikipedia/commons/6/69/Leiden_University_Library_-_Seikei_Zusetsu_vol._19%2C_page_019_-_%E7%8E%89%E8%9C%80%E9%BB%8D_-_Zea_mays_L.%2C_1804.jpg',
#     'Cytogenetics': 'https://upload.wikimedia.org/wikipedia/commons/a/ab/Angiosperm_Morphology_The_Closed_and_Collateral_Vascular_Bundle_in_Zea_%2837215761790%29.jpg',
#     'Evolution and Population Genetics': 'https://upload.wikimedia.org/wikipedia/commons/9/9a/CollapsedtreeLabels-simplified-fi.svg'
# }

In [49]:
# # Write poster presentation info ----

# for i in data['Posters'].index:
#     d = data['Posters'].loc[i, ].to_dict()

#     # if we have a multi-category case":
#     # import re
#     # categories = re.findall(r'>.+<', d['CATEGORY'])
#     # categories = [e.replace('>', '').replace('<', '').strip() for e in categories]
#     # categories

#     qmd_header = f"""---
# title: "{d['TITLE']}"
# subtitle: "{d['POSTER']}"
# author: "{d['PRESENTER_FIRST_NAME']+' '+d['PRESENTER_LAST_NAME']}"
# date: ""
# image: "{category_2_image[d['CATEGORY'].split('>')[1].split('<')[0].strip()]}"
# categories: 
# - {d['CATEGORY'].split('>')[1].split('<')[0].strip()}
# freeze: true
# editor: source
# markdown: 
#     wrap: 72
# ---"""

#     qmd_content = '\n\n'.join([
#         qmd_header,
#         d['ABSTRACT'],
#         d['AUTHOR_LIST'],
#         d['AFFILIATION_LIST']])

#     save_to = f"./generated/posters/{d['POSTER']}/index.qmd"

#     # make output dir if missing
#     safe_mkdir('/'.join(save_to.split('/')[0:-1]))

#     with open(save_to, 'w') as f:
#         f.writelines(qmd_content)

In [50]:
# # Write talk presentation info ----

# for i in data['Talks'].index:
#     d = data['Talks'].loc[i, ].to_dict()

#     qmd_header = f"""---
# title: "{d['TITLE']}"
# subtitle: "{d['TALK']}"
# author: "{d['PRESENTER_FIRST_NAME']+' '+d['PRESENTER_LAST_NAME']}"
# date: ""
# image: "{category_2_image[d['CATEGORY'].split('>')[1].split('<')[0].strip()]}"
# categories: 
# - {d['CATEGORY'].split('>')[1].split('<')[0].strip()}
# freeze: true
# editor: source
# markdown: 
#     wrap: 72
# ---"""

#     qmd_content = '\n\n'.join([
#         qmd_header,
#         d['ABSTRACT'],
#         d['AUTHOR_LIST'],
#         d['AFFILIATION_LIST']])

#     save_to = f"./generated/talks/{d['TALK']}/index.qmd"

#     # make output dir if missing
#     safe_mkdir('/'.join(save_to.split('/')[0:-1]))

#     with open(save_to, 'w') as f:
#         f.writelines(qmd_content)